In [1]:
!pip install fastparquet
!gdown --id '1cAthveg1d3MjrKJtMKGzfX3eH8HJ-dQp'
!gdown --id '1-RxLiAq8SBsKZECM7vno8eSazvZ-jMbG'
!unzip MedNLI_dataset.zip
!pip install medialpy
!pip install contractions

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 23.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 77.3 MB/s eta 0:00:00
/usr/local/lib/python3.10/dist-packages/gdown/__main__.py:132: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=1cAthveg1d3MjrKJtMKGzfX3eH8HJ-dQp
To: /content/MedNLI_dataset.zip
100% 681k/681k [00:00<00:00, 6.59MB/s]
/usr/local/lib/python3.10/dist-packages/gdown/__main__.py:132: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From (original): https://drive.google.com/uc?id=1-RxLiAq8SBsKZECM7vno8eSazvZ-jMbG
From (redirected): https://drive.google.com/uc?id=1-RxLiAq8SBsKZECM7vno8eSazvZ-jMbG&confirm=t&uuid=9692efbb-ec8f-497e-9826-8395eaffbe04
To: /content/m

In [2]:
from transformers import AutoTokenizer, AutoModel, AutoModelForSeq2SeqLM
import pandas as pd
import numpy as np
import time
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset,DataLoader, random_split
from sklearn.metrics import accuracy_score
import string
import medialpy
import contractions
#import shutil
#from google.colab import drive
#drive.mount('/content/drive')
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [3]:
test_data = pd.read_parquet('MedNLI_dataset/test-00000-of-00001-47685aa42db61e77.parquet', engine='fastparquet')
train_data = pd.read_parquet('MedNLI_dataset/train-00000-of-00001-210cfe9263b99806.parquet', engine='fastparquet')
valid_data = pd.read_parquet('MedNLI_dataset/valid-00000-of-00001-cc552de6d1a6fa4b.parquet', engine='fastparquet')

In [4]:
def find_pre_and_hyp(query):
    start_pre = query.find("[PRE]") + len("[PRE]")
    end_pre = query.find("[HYP]")
    start_hyp = query.find("[HYP]") + len("[HYP]")
    end_hyp = query.find("OUTPUT:")
    premise = query[start_pre:end_pre].strip()
    hypothesis = query[start_hyp:end_hyp].strip()

    return premise,hypothesis

In [5]:
class CustomDataset(Dataset):
    def __init__(self, x_list, y_list):
        self.samples = []
        for x,y in zip(x_list,y_list):
            #x_tensor = torch.tensor(x,dtype = torch.float32)
            y_tensor = torch.tensor(y,dtype = torch.float32)
            self.samples.append((x[0],x[1],y_tensor))

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        return self.samples[idx]

In [6]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
tokenizer = AutoTokenizer.from_pretrained("gsarti/biobert-nli")
bert_model = AutoModel.from_pretrained("gsarti/biobert-nli").to(device)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/136 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/454 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/433M [00:00<?, ?B/s]

In [7]:
def get_lists(data):
    x_list = []
    y_list = []
    i =0
    for query,answer in zip(data['query'],data['answer']):
        i = i + 1
        if answer == 'entailment':
           y = [1,0,0]
        elif answer == 'neutral':
           y = [0,1,0]
        elif answer == 'contradiction':
           y = [0,0,1]
        else:
           print('should not get here')

        premise,hypothesis = find_pre_and_hyp(query)
        x_list.append((premise,hypothesis))
        y_list.append(y)
    return x_list,y_list

In [8]:
train_x_list,train_y_list = get_lists(train_data)
test_x_list,test_y_list = get_lists(test_data)
val_x_list,val_y_list = get_lists(valid_data)

In [9]:
print(len(train_x_list))
print(len(train_y_list))
print(len(test_x_list))
print(len(test_y_list))
print(len(val_x_list))
print(len(val_y_list))

11232
11232
1422
1422
1395
1395


In [10]:
train_dataset = CustomDataset(train_x_list,train_y_list)
test_dataset = CustomDataset(test_x_list,test_y_list)
val_dataset = CustomDataset(val_x_list,val_y_list)

In [11]:
class embedding_layer(nn.Module):
  def __init__(self,bert_model,tokenizer):
    super(embedding_layer, self).__init__()
    self.bert_model = bert_model
    self.tokenizer = tokenizer

  def forward(self, x):
    with torch.no_grad():
         tokens = tokenizer(x,return_tensors="pt",padding=True).to(device)
         we = bert_model(**tokens)['last_hidden_state'].to(device)
    return we

In [12]:
class PositionalEncoding(nn.Module):
    def __init__(self,embedding_size):
        super().__init__()
        self.dropout = nn.Dropout(0.1)
        self.embedding_size = embedding_size

    def forward(self, x):
        pe = torch.zeros(x.size(0), x.size(1), self.embedding_size).to(device)
        div_term = torch.zeros(x.size(0), 1, self.embedding_size).to(device)
        ks = torch.arange(self.embedding_size).float().to(device)
        values = torch.exp(-torch.log(torch.tensor(1000.0)) * 2 * ks / self.embedding_size).to(device)
        values = values.view(1, 1, -1).to(device)
        div_term = div_term + values
        x = x.reshape([x.shape[0],x.shape[1],1]).to(device)
        pe[:, :, ::2] = torch.sin(x * div_term)[:, :, ::2].to(device)
        pe[:, :, 1::2] = torch.cos(x * div_term)[:, :, 1::2].to(device)
        pe = self.dropout(pe)
        return pe

In [13]:
class MultiHeadAttention(nn.Module):
    def __init__(self, embedding_size, heads):
        super().__init__()
        self.embedding_size = embedding_size
        self.heads = heads
        self.head_dim = embedding_size // heads
        assert(self.heads * self.head_dim == self.embedding_size), "Invalid number of heads"
        self.fc_values = nn.Linear(self.head_dim, self.head_dim, bias=False).to(device)
        self.fc_keys = nn.Linear(self.head_dim, self.head_dim, bias=False).to(device)
        self.fc_queries = nn.Linear(self.head_dim, self.head_dim, bias=False).to(device)
        self.fc_out = nn.Linear(heads * self.head_dim, embedding_size).to(device)

    def forward(self, values, keys, queries):
        N = queries.shape[0]
        value_len, key_len, query_len = values.shape[1], keys.shape[1], queries.shape[1]
        values = values.reshape(N, value_len, self.heads, self.head_dim)
        keys = keys.reshape(N, key_len, self.heads, self.head_dim)
        queries = queries.reshape(N, query_len, self.heads, self.head_dim)
        values = self.fc_values(values).to(device)
        keys = self.fc_keys(keys).to(device)
        queries = self.fc_queries(queries).to(device)
        energy = torch.einsum("nqhd,nkhd->nhqk", [queries, keys]).to(device)
        energy = torch.softmax(energy / (self.embedding_size ** 0.5), dim=3).to(device)
        attention = torch.einsum("nhql,nlhd->nqhd", [energy, values]).to(device)
        attention = attention.reshape(N, query_len, self.heads * self.head_dim).to(device)
        x = self.fc_out(attention)
        return x

In [14]:
class TransformerBlock(nn.Module):
    def __init__(self, embedding_size, heads, forward_expansion, p):
        super().__init__()
        self.attention = MultiHeadAttention(embedding_size, heads)
        self.norm1 = nn.LayerNorm(embedding_size)
        self.feed_forward = nn.Sequential(nn.Linear(embedding_size, forward_expansion * embedding_size),
                                          nn.ReLU(),
                                          nn.Linear(forward_expansion * embedding_size, embedding_size))
        self.norm2 = nn.LayerNorm(embedding_size)
        self.dropout = nn.Dropout(p)
    def forward(self, values, keys, queries):
        attention_out = self.attention(values, keys, queries)
        x = self.norm1(attention_out + queries)
        x = self.dropout(x)
        x = self.norm2(self.feed_forward(x) + x)
        x = self.dropout(x)
        return x

In [15]:
class Encoder(nn.Module):
    def __init__(self, embedding_size,num_layers, forward_expansion, heads, p, device):
        super().__init__()
        self.device = device
        self.word_embedding = embedding_layer(bert_model,tokenizer)
        self.positional_embedding = PositionalEncoding(embedding_size)
        self.layers = nn.ModuleList([TransformerBlock(embedding_size, heads, forward_expansion, p) for _ in range(num_layers)])
        self.dropout = nn.Dropout(p)

    def forward(self, x):
        we = self.word_embedding(x)
        positions = torch.arange(0, we.size(1)).expand(we.size(0), we.size(1)).to(self.device)
        pe = self.positional_embedding(positions)
        x = self.dropout(we + pe)
        for layer in self.layers:
            x = layer(x, x, x)
        return x
        return we

In [16]:
class Transformer(nn.Module):
    def __init__(self, embedding_size=768,
                 num_layers=4, forward_expansion=8, heads=8, p=0.1):
        super().__init__()
        self.encoder = Encoder(embedding_size, num_layers, heads,
                               forward_expansion, p, device)

    def forward(self, src):
        enc_out = self.encoder(src).to(device)
        return enc_out

In [17]:
class NN(nn.Module):
    def __init__(self):
        super(NN, self).__init__()
        self.dropout1 = nn.Dropout(0.1)
        self.dropout2 = nn.Dropout(0.1)
        self.transformer1 = Transformer().to(device)
        self.transformer2 = Transformer().to(device)
        self.fc1 = nn.Linear(768, 256).to(device)
        self.fc2 = nn.Linear(256, 3).to(device)

    def deabbreviation(self,text):
        try:
           return medialpy.find(text).meaning[0]
        except:
           return text

    def preprocess_text(self,text):
        text = contractions.fix(text)
        text = text.translate(str.maketrans('', '', string.punctuation))
        tokens = word_tokenize(text)
        ntokens = []
        for token in tokens:
            ntokens.append(self.deabbreviation(token).lower())
        stop_words = set(stopwords.words('english'))
        stop_words.remove('no')
        stop_words.remove('not')
        filtered_tokens = [word for word in ntokens if word not in stop_words]
        processed_text = ' '.join(filtered_tokens)
        return processed_text

    def forward(self, x_pre,x_hyp):
        nx_pre = ()
        nx_hyp = ()
        for t in x_pre:
            nx_pre += (self.preprocess_text(t),)
        for t in x_hyp:
            nx_hyp += (self.preprocess_text(t),)
        x_pre = nx_pre
        x_hyp = nx_hyp
        enc_x_pre = self.transformer1(x_pre)
        enc_x_hyp = self.transformer2(x_hyp)
        enc_x_pre = torch.mean(enc_x_pre,1)
        enc_x_hyp = torch.mean(enc_x_hyp,1)
        x = enc_x_pre * enc_x_hyp
        x = F.relu(self.fc1(self.dropout1(x)))
        x = nn.functional.softmax(self.fc2(self.dropout2(x)),dim=1)
        return x

In [18]:
bsize = 32
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=bsize, shuffle=True)
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=bsize, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=bsize, shuffle=False)

In [24]:
model = NN()
#model = torch.load('modeln.pth')
criterion = nn.CrossEntropyLoss()
optimizer = optim.AdamW(model.parameters(), lr=0.00001)

In [25]:
for param in model.transformer1.encoder.word_embedding.parameters():
    param.requires_grad = False
for param in model.transformer2.encoder.word_embedding.parameters():
    param.requires_grad = False

In [26]:
# Calculate and print the number of parameters along with their names
#for name, param in model.named_parameters():
    #print(f"{name}: {param.numel()} parameters")
    #print(param.requires_grad)
    #print('======================================================================')

# Total number of parameters
total_params = sum(param.numel() for param in model.parameters() if param.requires_grad)
print(f'Total number of parameters: {total_params}')

Total number of parameters: 80720899


In [27]:
def get_model_acc(model,data_loader):
    predictions = []
    true_labels = []
    for x_pre, x_hyp, y in data_loader:
        outputs = model(x_pre,x_hyp)
        predicted_labels = torch.argmax(outputs, dim=1).cpu().numpy()
        predictions.extend(predicted_labels)
        true_label = torch.argmax(y, dim=1).cpu().numpy()
        true_labels.extend(true_label)
    return accuracy_score(true_labels, predictions)

In [28]:
model.train()
num_epochs = 2
for epoch in range(num_epochs):
    for x_pre,x_hyp,y in train_loader:
        optimizer.zero_grad()
        outputs = model(x_pre,x_hyp)
        y = y.to(device)
        loss = criterion(outputs, y)
        loss.backward()
        optimizer.step()
    #print(get_model_acc(model,train_loader))
    with torch.no_grad():
         model.eval()
         acc = get_model_acc(model,test_loader)
         model.train()
    print(acc)
    print(loss.item())
    print('===========================================================================')
print('Training completed.')

0.5857946554149086
0.958264172077179
0.6568213783403657
0.8532441854476929
Training completed.
